# MNIST 모델 학습 구현하기
## 1. Activation Function으로 Sigmoid 사용
### 1) import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist # fashion_mnist, cifar10, cifar100 등의 dataset도 있음
from time import time
import os
print(tf.__version__)

2.3.0


### 2) Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(' [*] Reading checkpoints...')
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(' [*] Success to read {}'.format(ckpt_name))
        return True, counter
    else:
        print(' [*] Failed to a checkpoint')
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### 3) Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data() # train 이미지 : 60,000장 / test 이미지 : 10,000장
    # 차원을 바꾸는 이유 : tensorflow가 data를 사용할 때 [batch_size, height, width, channel]로 받기 때문
    # MNIST의 경우 gray scale로 되어 있어, channel 값이 생략 되어 있기 때문에 이를 추가해줘야 한다.
    # axis : 차원을 늘릴 위치 -> -1은 오른쪽 끝을 의미함(index 개념으로 이해하면 쉬울 듯)
    train_data = np.expand_dims(train_data, axis=-1) # 차원 추가 # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # 차원 추가 # [N, 28, 28] -> [N, 28, 28, 1]
    
    # normalization
    train_data, test_data = normalize(train_data, test_data)
    
    # to_categorical : 우리가 분류할 label의 갯수를 추가해준다.
    # One hot Encoding : 기존의 이미지를 카테고리화 하여 해당 인덱스에 값을 1로 변경하는 Encoding
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]
    return train_data, train_labels, test_data, test_labels

# normalize : 이미지 값은 [0, 255]의 값을 갖는데, 이것을 [0, 1]의 범위로 바꿔줘야 함
# astype : 데이터 프레임의 데이터 타입 바꾸기(pandas)
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

### 4) Performance function
- `tf.argmax` : 가장 큰 숫자(여기서는 One hot에 대한 probability)의 위치(index) 반환

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### 5) Model function
- sigmoid() 함수를 만듦

In [5]:
# flatten : shape 펼치기
def flatten():
    return tf.keras.layers.Flatten()

# dense layer (= fully connected layer)
# 앞 Layer의 모든 뉴런과 연결이 되어 있는 Layer를 의미
def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

### 6) Create model(class)
- activation function으로 sigmoid를 쓴다.
- class로 모델을 만들 땐 꼭 `tf.keras.Model`을 상속 받아야 한다.

In [6]:
class Create_model_class(tf.keras.Model):
    def __init__(self, lable_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential() # Sequential : List 자료구조와 비슷
        self.model.add(flatten()) # [N, 28, 28, 1] -> [N, 784]
        
        for i in range(2):
            # [N, 784] -> [N, 256] -> [N, 256]
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())
            
        self.model.add(dense(label_dim, weight_init)) # [N, 256] -> [N, 10]
    
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### 7) Create model(function)
- activation function으로 sigmoid를 쓴다.
- class와 function의 성능 차이는 거의 없다

In [7]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256, weight_init))
        model.add(sigmoid())

    model.add(dense(label_dim, weight_init))
    
    return model

### 8) Define data & hyper-parameter
- dataset shuffling할 때 buffer_size는 dataset 크기보다 큰 수를 적어주면 랜덤하게 shuffle된다.

In [8]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

# Graph Input using Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### 9) Define model & optimizer & writer
- function으로 구성한 model을 사용

In [9]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### 10) Restore checkpoint & start train or test phase
- Checkpoint의 역할 : 학습하다가 네트워크가 끊겼을 때, 재학습을 하기 위해 변경된 weight를 불러낸다.

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)
    # global_step = tf.train.create_global_step()

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    # _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8160, train_loss: 2.26998377, train_accuracy: 0.1406, test_Accuracy: 0.1032
Epoch: [ 0] [    1/  468] time: 1.1850, train_loss: 2.30493259, train_accuracy: 0.0859, test_Accuracy: 0.1053
Epoch: [ 0] [    2/  468] time: 1.5050, train_loss: 2.27619410, train_accuracy: 0.0938, test_Accuracy: 0.1009
Epoch: [ 0] [    3/  468] time: 1.8210, train_loss: 2.28575158, train_accuracy: 0.1328, test_Accuracy: 0.1009
Epoch: [ 0] [    4/  468] time: 2.1710, train_loss: 2.23631239, train_accuracy: 0.1250, test_Accuracy: 0.1011
Epoch: [ 0] [    5/  468] time: 2.4930, train_loss: 2.27999878, train_accuracy: 0.1016, test_Accuracy: 0.1211
Epoch: [ 0] [    6/  468] time: 2.8120, train_loss: 2.29508686, train_accuracy: 0.0469, test_Accuracy: 0.1011
Epoch: [ 0] [    7/  468] time: 3.1220, train_loss: 2.24555826, train_accuracy: 0.0703, test_Accuracy: 0.1083
Epoch: [ 0] [    8/  468] time: 3.4180, trai

Epoch: [ 0] [   74/  468] time: 23.7400, train_loss: 0.98621303, train_accuracy: 0.8047, test_Accuracy: 0.7809
Epoch: [ 0] [   75/  468] time: 24.0470, train_loss: 0.90173578, train_accuracy: 0.8047, test_Accuracy: 0.7737
Epoch: [ 0] [   76/  468] time: 24.3370, train_loss: 0.90261561, train_accuracy: 0.7812, test_Accuracy: 0.7703
Epoch: [ 0] [   77/  468] time: 24.6270, train_loss: 0.97305042, train_accuracy: 0.7266, test_Accuracy: 0.7710
Epoch: [ 0] [   78/  468] time: 24.9120, train_loss: 0.96333063, train_accuracy: 0.7344, test_Accuracy: 0.7789
Epoch: [ 0] [   79/  468] time: 25.1850, train_loss: 0.90306306, train_accuracy: 0.7578, test_Accuracy: 0.7928
Epoch: [ 0] [   80/  468] time: 25.4970, train_loss: 0.87802887, train_accuracy: 0.8203, test_Accuracy: 0.8049
Epoch: [ 0] [   81/  468] time: 25.8160, train_loss: 0.88697606, train_accuracy: 0.7891, test_Accuracy: 0.8111
Epoch: [ 0] [   82/  468] time: 26.1110, train_loss: 0.84810102, train_accuracy: 0.8125, test_Accuracy: 0.8145
E

Epoch: [ 0] [  148/  468] time: 50.6910, train_loss: 0.50810957, train_accuracy: 0.9062, test_Accuracy: 0.8746
Epoch: [ 0] [  149/  468] time: 50.9720, train_loss: 0.46321285, train_accuracy: 0.8594, test_Accuracy: 0.8746
Epoch: [ 0] [  150/  468] time: 51.2670, train_loss: 0.49777949, train_accuracy: 0.8594, test_Accuracy: 0.8750
Epoch: [ 0] [  151/  468] time: 51.5490, train_loss: 0.59454405, train_accuracy: 0.8281, test_Accuracy: 0.8771
Epoch: [ 0] [  152/  468] time: 51.8250, train_loss: 0.44040361, train_accuracy: 0.8984, test_Accuracy: 0.8798
Epoch: [ 0] [  153/  468] time: 52.1010, train_loss: 0.55163407, train_accuracy: 0.8828, test_Accuracy: 0.8820
Epoch: [ 0] [  154/  468] time: 52.3840, train_loss: 0.39933610, train_accuracy: 0.8984, test_Accuracy: 0.8832
Epoch: [ 0] [  155/  468] time: 52.6610, train_loss: 0.56484830, train_accuracy: 0.8594, test_Accuracy: 0.8832
Epoch: [ 0] [  156/  468] time: 52.9470, train_loss: 0.47998706, train_accuracy: 0.9141, test_Accuracy: 0.8820
E

Epoch: [ 0] [  222/  468] time: 72.8930, train_loss: 0.42561156, train_accuracy: 0.8828, test_Accuracy: 0.9067
Epoch: [ 0] [  223/  468] time: 73.1680, train_loss: 0.43638974, train_accuracy: 0.8828, test_Accuracy: 0.9068
Epoch: [ 0] [  224/  468] time: 73.4460, train_loss: 0.24970859, train_accuracy: 0.9609, test_Accuracy: 0.9071
Epoch: [ 0] [  225/  468] time: 73.7320, train_loss: 0.43305606, train_accuracy: 0.8750, test_Accuracy: 0.9069
Epoch: [ 0] [  226/  468] time: 74.0050, train_loss: 0.46790257, train_accuracy: 0.8828, test_Accuracy: 0.9061
Epoch: [ 0] [  227/  468] time: 74.2810, train_loss: 0.23377131, train_accuracy: 0.9688, test_Accuracy: 0.9052
Epoch: [ 0] [  228/  468] time: 74.5550, train_loss: 0.41382515, train_accuracy: 0.9062, test_Accuracy: 0.9052
Epoch: [ 0] [  229/  468] time: 74.8310, train_loss: 0.51518220, train_accuracy: 0.8672, test_Accuracy: 0.9043
Epoch: [ 0] [  230/  468] time: 75.1200, train_loss: 0.31252468, train_accuracy: 0.9141, test_Accuracy: 0.9036
E

Epoch: [ 0] [  296/  468] time: 93.4830, train_loss: 0.44521025, train_accuracy: 0.8594, test_Accuracy: 0.9126
Epoch: [ 0] [  297/  468] time: 93.7640, train_loss: 0.32763493, train_accuracy: 0.9062, test_Accuracy: 0.9121
Epoch: [ 0] [  298/  468] time: 94.0440, train_loss: 0.28678751, train_accuracy: 0.9141, test_Accuracy: 0.9133
Epoch: [ 0] [  299/  468] time: 94.3180, train_loss: 0.26455781, train_accuracy: 0.9062, test_Accuracy: 0.9124
Epoch: [ 0] [  300/  468] time: 94.5940, train_loss: 0.22765104, train_accuracy: 0.9375, test_Accuracy: 0.9111
Epoch: [ 0] [  301/  468] time: 94.9260, train_loss: 0.31292930, train_accuracy: 0.9297, test_Accuracy: 0.9117
Epoch: [ 0] [  302/  468] time: 95.2220, train_loss: 0.30399516, train_accuracy: 0.8828, test_Accuracy: 0.9122
Epoch: [ 0] [  303/  468] time: 95.4990, train_loss: 0.26019502, train_accuracy: 0.9219, test_Accuracy: 0.9134
Epoch: [ 0] [  304/  468] time: 95.7980, train_loss: 0.30076277, train_accuracy: 0.8984, test_Accuracy: 0.9144
E

Epoch: [ 0] [  370/  468] time: 114.1360, train_loss: 0.29191798, train_accuracy: 0.9062, test_Accuracy: 0.9171
Epoch: [ 0] [  371/  468] time: 114.4100, train_loss: 0.28401068, train_accuracy: 0.8828, test_Accuracy: 0.9175
Epoch: [ 0] [  372/  468] time: 114.6800, train_loss: 0.23288113, train_accuracy: 0.9375, test_Accuracy: 0.9193
Epoch: [ 0] [  373/  468] time: 114.9590, train_loss: 0.30885401, train_accuracy: 0.9219, test_Accuracy: 0.9196
Epoch: [ 0] [  374/  468] time: 115.2380, train_loss: 0.28073293, train_accuracy: 0.9062, test_Accuracy: 0.9202
Epoch: [ 0] [  375/  468] time: 115.5240, train_loss: 0.14873594, train_accuracy: 0.9766, test_Accuracy: 0.9196
Epoch: [ 0] [  376/  468] time: 115.7950, train_loss: 0.34387171, train_accuracy: 0.9141, test_Accuracy: 0.9197
Epoch: [ 0] [  377/  468] time: 116.0690, train_loss: 0.38519081, train_accuracy: 0.9062, test_Accuracy: 0.9194
Epoch: [ 0] [  378/  468] time: 116.3420, train_loss: 0.28766069, train_accuracy: 0.9375, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 134.6220, train_loss: 0.30683404, train_accuracy: 0.9062, test_Accuracy: 0.9222
Epoch: [ 0] [  445/  468] time: 134.9060, train_loss: 0.20507209, train_accuracy: 0.9531, test_Accuracy: 0.9229
Epoch: [ 0] [  446/  468] time: 135.1760, train_loss: 0.41191360, train_accuracy: 0.9062, test_Accuracy: 0.9231
Epoch: [ 0] [  447/  468] time: 135.4520, train_loss: 0.29450011, train_accuracy: 0.9062, test_Accuracy: 0.9225
Epoch: [ 0] [  448/  468] time: 135.7240, train_loss: 0.22903027, train_accuracy: 0.9141, test_Accuracy: 0.9208
Epoch: [ 0] [  449/  468] time: 135.9970, train_loss: 0.19946027, train_accuracy: 0.9375, test_Accuracy: 0.9199
Epoch: [ 0] [  450/  468] time: 136.2660, train_loss: 0.22947237, train_accuracy: 0.9375, test_Accuracy: 0.9200
Epoch: [ 0] [  451/  468] time: 136.5360, train_loss: 0.23316474, train_accuracy: 0.9219, test_Accuracy: 0.9199
Epoch: [ 0] [  452/  468] time: 136.8070, train_loss: 0.27376127, train_accuracy: 0.9453, test_Accuracy:

## 2. Activation Function으로 ReLU 사용
### 5) Model Function
- activation function을 ReLU로 구성

In [11]:
def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

### 6) Create Model(class vision)
- ReLU를 activation function로 사용

In [12]:
class Create_model_class(tf.keras.Model):
    def __init__(self, lable_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequentail()
        self.model.add(flatten())
        
        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu()) # ReLU 추가
            
        self.model.add(dense(label_dim, weight_init))
    
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### 7) Create Model(function vision)
- ReLU를 activation function으로 사용

In [13]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256, weight_init))
        model.add(relu()) # ReLU 추가

    model.add(dense(label_dim, weight_init))
    
    return model

### 9) Define Model & optimizer & writer
- sigmoid 예제와 다른 경로의 폴더를 지정

In [14]:
# Model
# create_model_function을 재정의 했으므로 network 변수에 새로 assign
network = create_model_function(label_dim)

# Training """
# optimizer는 위에 선언했으므로 생략

# Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu' # 디렉토리 명 바꿔주기

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### 10) Restore checkpoint & start train or test phase

In [15]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.7630, train_loss: 2.07429838, train_accuracy: 0.5547, test_Accuracy: 0.4009
Epoch: [ 0] [    1/  468] time: 1.1100, train_loss: 1.88599372, train_accuracy: 0.5859, test_Accuracy: 0.5045
Epoch: [ 0] [    2/  468] time: 1.3890, train_loss: 1.66146874, train_accuracy: 0.6562, test_Accuracy: 0.5973
Epoch: [ 0] [    3/  468] time: 1.6670, train_loss: 1.45093322, train_accuracy: 0.7031, test_Accuracy: 0.6748
Epoch: [ 0] [    4/  468] time: 1.9450, train_loss: 1.14773130, train_accuracy: 0.7109, test_Accuracy: 0.7277
Epoch: [ 0] [    5/  468] time: 2.2400, train_loss: 0.81169260, train_accuracy: 0.8281, test_Accuracy: 0.7405
Epoch: [ 0] [    6/  468] time: 2.5160, train_loss: 0.69884366, train_accuracy: 0.8125, test_Accuracy: 0.7674
Epoch: [ 0] [    7/  468] time: 2.7970, train_loss: 0.65336609, train_accuracy: 0.7734, test_Accuracy: 0.7029
Epoch: [ 0] [    8/  468] time: 3.0770, trai

Epoch: [ 0] [   74/  468] time: 21.2880, train_loss: 0.24535033, train_accuracy: 0.9062, test_Accuracy: 0.9226
Epoch: [ 0] [   75/  468] time: 21.5670, train_loss: 0.39398777, train_accuracy: 0.8828, test_Accuracy: 0.9234
Epoch: [ 0] [   76/  468] time: 21.8410, train_loss: 0.19835472, train_accuracy: 0.9453, test_Accuracy: 0.9263
Epoch: [ 0] [   77/  468] time: 22.1100, train_loss: 0.17674062, train_accuracy: 0.9375, test_Accuracy: 0.9253
Epoch: [ 0] [   78/  468] time: 22.3810, train_loss: 0.32107931, train_accuracy: 0.9062, test_Accuracy: 0.9233
Epoch: [ 0] [   79/  468] time: 22.6550, train_loss: 0.28210923, train_accuracy: 0.9141, test_Accuracy: 0.9227
Epoch: [ 0] [   80/  468] time: 22.9250, train_loss: 0.11729037, train_accuracy: 0.9766, test_Accuracy: 0.9228
Epoch: [ 0] [   81/  468] time: 23.1950, train_loss: 0.18805300, train_accuracy: 0.9531, test_Accuracy: 0.9236
Epoch: [ 0] [   82/  468] time: 23.4720, train_loss: 0.21704265, train_accuracy: 0.9297, test_Accuracy: 0.9262
E

Epoch: [ 0] [  148/  468] time: 41.6990, train_loss: 0.17003307, train_accuracy: 0.9297, test_Accuracy: 0.9456
Epoch: [ 0] [  149/  468] time: 41.9770, train_loss: 0.19806370, train_accuracy: 0.9531, test_Accuracy: 0.9476
Epoch: [ 0] [  150/  468] time: 42.2520, train_loss: 0.10670182, train_accuracy: 0.9531, test_Accuracy: 0.9473
Epoch: [ 0] [  151/  468] time: 42.5250, train_loss: 0.10878763, train_accuracy: 0.9688, test_Accuracy: 0.9463
Epoch: [ 0] [  152/  468] time: 42.8020, train_loss: 0.14849883, train_accuracy: 0.9609, test_Accuracy: 0.9451
Epoch: [ 0] [  153/  468] time: 43.0900, train_loss: 0.17724232, train_accuracy: 0.9609, test_Accuracy: 0.9448
Epoch: [ 0] [  154/  468] time: 43.3860, train_loss: 0.26316169, train_accuracy: 0.9141, test_Accuracy: 0.9469
Epoch: [ 0] [  155/  468] time: 43.6860, train_loss: 0.19472280, train_accuracy: 0.9297, test_Accuracy: 0.9482
Epoch: [ 0] [  156/  468] time: 43.9610, train_loss: 0.15433872, train_accuracy: 0.9531, test_Accuracy: 0.9487
E

Epoch: [ 0] [  222/  468] time: 62.2950, train_loss: 0.11908643, train_accuracy: 0.9609, test_Accuracy: 0.9500
Epoch: [ 0] [  223/  468] time: 62.5740, train_loss: 0.12591153, train_accuracy: 0.9766, test_Accuracy: 0.9514
Epoch: [ 0] [  224/  468] time: 62.8450, train_loss: 0.21414685, train_accuracy: 0.9297, test_Accuracy: 0.9529
Epoch: [ 0] [  225/  468] time: 63.1160, train_loss: 0.17491865, train_accuracy: 0.9375, test_Accuracy: 0.9523
Epoch: [ 0] [  226/  468] time: 63.3880, train_loss: 0.10504852, train_accuracy: 0.9766, test_Accuracy: 0.9535
Epoch: [ 0] [  227/  468] time: 63.6700, train_loss: 0.23100489, train_accuracy: 0.9531, test_Accuracy: 0.9530
Epoch: [ 0] [  228/  468] time: 63.9430, train_loss: 0.10994501, train_accuracy: 0.9844, test_Accuracy: 0.9538
Epoch: [ 0] [  229/  468] time: 64.2140, train_loss: 0.16192722, train_accuracy: 0.9219, test_Accuracy: 0.9539
Epoch: [ 0] [  230/  468] time: 64.5030, train_loss: 0.10089006, train_accuracy: 0.9766, test_Accuracy: 0.9542
E

Epoch: [ 0] [  296/  468] time: 84.0510, train_loss: 0.26475826, train_accuracy: 0.9297, test_Accuracy: 0.9624
Epoch: [ 0] [  297/  468] time: 84.3270, train_loss: 0.12347306, train_accuracy: 0.9688, test_Accuracy: 0.9620
Epoch: [ 0] [  298/  468] time: 84.6220, train_loss: 0.10363759, train_accuracy: 0.9844, test_Accuracy: 0.9619
Epoch: [ 0] [  299/  468] time: 84.9180, train_loss: 0.17982599, train_accuracy: 0.9219, test_Accuracy: 0.9604
Epoch: [ 0] [  300/  468] time: 85.2400, train_loss: 0.12527838, train_accuracy: 0.9609, test_Accuracy: 0.9604
Epoch: [ 0] [  301/  468] time: 85.5130, train_loss: 0.20441552, train_accuracy: 0.9375, test_Accuracy: 0.9568
Epoch: [ 0] [  302/  468] time: 85.7890, train_loss: 0.09906834, train_accuracy: 0.9766, test_Accuracy: 0.9548
Epoch: [ 0] [  303/  468] time: 86.0830, train_loss: 0.13738075, train_accuracy: 0.9688, test_Accuracy: 0.9541
Epoch: [ 0] [  304/  468] time: 86.3670, train_loss: 0.12094542, train_accuracy: 0.9688, test_Accuracy: 0.9556
E

Epoch: [ 0] [  370/  468] time: 107.0620, train_loss: 0.13589916, train_accuracy: 0.9609, test_Accuracy: 0.9648
Epoch: [ 0] [  371/  468] time: 107.3350, train_loss: 0.12628411, train_accuracy: 0.9531, test_Accuracy: 0.9642
Epoch: [ 0] [  372/  468] time: 107.6120, train_loss: 0.06856550, train_accuracy: 0.9766, test_Accuracy: 0.9641
Epoch: [ 0] [  373/  468] time: 107.8880, train_loss: 0.11834675, train_accuracy: 0.9688, test_Accuracy: 0.9649
Epoch: [ 0] [  374/  468] time: 108.1640, train_loss: 0.19464777, train_accuracy: 0.9375, test_Accuracy: 0.9654
Epoch: [ 0] [  375/  468] time: 108.4480, train_loss: 0.16396092, train_accuracy: 0.9453, test_Accuracy: 0.9650
Epoch: [ 0] [  376/  468] time: 108.7250, train_loss: 0.10447516, train_accuracy: 0.9609, test_Accuracy: 0.9652
Epoch: [ 0] [  377/  468] time: 109.0030, train_loss: 0.08550338, train_accuracy: 0.9609, test_Accuracy: 0.9653
Epoch: [ 0] [  378/  468] time: 109.2740, train_loss: 0.17643161, train_accuracy: 0.9375, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 127.6020, train_loss: 0.12826732, train_accuracy: 0.9766, test_Accuracy: 0.9676
Epoch: [ 0] [  445/  468] time: 127.8730, train_loss: 0.07461514, train_accuracy: 0.9844, test_Accuracy: 0.9674
Epoch: [ 0] [  446/  468] time: 128.1470, train_loss: 0.05459716, train_accuracy: 0.9844, test_Accuracy: 0.9675
Epoch: [ 0] [  447/  468] time: 128.4290, train_loss: 0.03933295, train_accuracy: 0.9922, test_Accuracy: 0.9674
Epoch: [ 0] [  448/  468] time: 128.7000, train_loss: 0.04082289, train_accuracy: 0.9844, test_Accuracy: 0.9671
Epoch: [ 0] [  449/  468] time: 128.9700, train_loss: 0.10444099, train_accuracy: 0.9531, test_Accuracy: 0.9678
Epoch: [ 0] [  450/  468] time: 129.2530, train_loss: 0.09589690, train_accuracy: 0.9844, test_Accuracy: 0.9680
Epoch: [ 0] [  451/  468] time: 129.5250, train_loss: 0.05526493, train_accuracy: 0.9766, test_Accuracy: 0.9684
Epoch: [ 0] [  452/  468] time: 129.8080, train_loss: 0.12503491, train_accuracy: 0.9688, test_Accuracy:

## 3. 결론
> ReLU를 쓸 때 보다 높은 accuracy의 모델을 만들 수 있다.

### Sigmoid
- Test accuracy : 92.28%
### ReLU
- Test accuracy : 96.69%